In [1]:
from torch.utils.data import DataLoader
import json

from rouge_score import rouge_scorer
from models.models import Summarizer
from summarizer import Summarizer
# import torch

/home/sayyor/miniconda3/envs/nlp/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt to /home/sayyor/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to /home/sayyor/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
# Load the model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Summarizer(device)
model.load_state_dict(torch.load('output_fullarxiv/model_linear.pt'))
model.to(device)
model.eval()

Summarizer(
  (encoder): EncoderModel(
    (encoder): DistilBertModel(
      (embeddings): Embeddings(
        (word_embeddings): Embedding(30522, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (transformer): Transformer(
        (layer): ModuleList(
          (0-5): 6 x TransformerBlock(
            (attention): MultiHeadSelfAttention(
              (dropout): Dropout(p=0.1, inplace=False)
              (q_lin): Linear(in_features=768, out_features=768, bias=True)
              (k_lin): Linear(in_features=768, out_features=768, bias=True)
              (v_lin): Linear(in_features=768, out_features=768, bias=True)
              (out_lin): Linear(in_features=768, out_features=768, bias=True)
            )
            (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (ffn): FFN(
              (dr

### Testing with chunks

In [3]:
test_location = "preprocessing/data/arxiv_summarization/stories/test.pt"

# Load the test data
test_data = torch.load(test_location)

sample = test_data['test_2_0']

print(sample['article'])
print(sample['abstract'])

# Generate the summary
summary, _ = model.predict(article,5, method='sum_all')
print(summary)

# Compute the ROUGE scores
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

abstract = ' '.join(sample['abstract'])
scores_2 = scorer.score(abstract, summary)

print(scores_2)

The second approach, $take\_N\_sents$, involved extracting the best N summary sentences from each chunk and joining them together. ROUGE-2: measures the overlap of bigrams (two-word sequences) between the generated text, and the reference text. The tokens are converted to an embedding space. There are two approaches for this task: extractive and abstractive summarization. Summarization is a popular NLP task, since the ability to summarize a text is inherently human and important for text understanding.


### Testing with the full text

In [4]:
# Test for the full text
test_location = '/home/sayyor/Desktop/BDMA_Studies/Machine Learning/NLP_Paper_Summarizer/arxiv_files/stories_full/test.json'
with open(test_location) as f:
    data = json.load(f)

sample = data['test_0']

print(sample['article'])
print(sample['abstract'])

# Generate the summary
summary, _ = model.predict(sample['article'], sum_len=len(sample['abstract']))

# Generate the summary of getting N-top
summary2, _ = model.predict(sample['article'], sum_len=len(sample['abstract']), method='takeN')


# Compute the ROUGE scores
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

abstract = ' '.join(sample['abstract'])
scores_1 = scorer.score(abstract, summary)
scores_2 = scorer.score(abstract, summary2)

print('sum_all', scores_1)
print('takeN', scores_2)

for about 20 years the problem of properties of short - term changes of solar activity has been considered extensively . 
 many investigators studied the short - term periodicities of the various indices of solar activity . 
 several periodicities were detected , but the periodicities about 155 days and from the interval of @xmath3 $ ] days ( @xmath4 $ ] years ) are mentioned most often . 
 first of them was discovered by @xcite in the occurence rate of gamma - ray flares detected by the gamma - ray spectrometer aboard the _ solar maximum mission ( smm ) . 
 this periodicity was confirmed for other solar flares data and for the same time period @xcite . 
 it was also found in proton flares during solar cycles 19 and 20 @xcite , but it was not found in the solar flares data during solar cycles 22 @xcite . 
 _    several autors confirmed above results for the daily sunspot area data . @xcite studied the sunspot data from 18741984 . 
 she found the 155-day periodicity in data records from

In [21]:
print("sum_all")
for key, value in scores_1.items():
    print(key)
    values = list(scores_1[key])
    print('\t'+'precision: '+str(round(values[0],3)))
    print('\t'+'recall: '+str(round(values[1],3)))
    print('\t'+'fmeasure: '+str(round(values[2],3)))
print("")
print("takeN")
for key, value in scores_2.items():
    print(key)
    values = list(scores_2[key])
    print('\t'+'precision: '+str(round(values[0],3)))
    print('\t'+'recall: '+str(round(values[1],3)))
    print('\t'+'fmeasure: '+str(round(values[2],3)))

sum_all
rouge1
	precision: 0.459
	recall: 0.359
	fmeasure: 0.403
rouge2
	precision: 0.235
	recall: 0.183
	fmeasure: 0.206
rougeL
	precision: 0.233
	recall: 0.182
	fmeasure: 0.205

takeN
rouge1
	precision: 0.658
	recall: 0.147
	fmeasure: 0.24
rouge2
	precision: 0.297
	recall: 0.065
	fmeasure: 0.107
rougeL
	precision: 0.395
	recall: 0.088
	fmeasure: 0.144


### Running original Bert Summarize for all test data

In [2]:
from summarizer import Summarizer
from rouge_score import rouge_scorer
from tqdm import tqdm

test_location = "preprocessing/data/arxiv_summarization/stories/test.pt"
# Load the test data
test_data = torch.load(test_location)

model_orig = Summarizer()

summaries = {}
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeLsum'])
scores = {}
scores['rouge1'] = []
scores['rouge2'] = []
scores['rougeLsum'] = []
for key in tqdm(test_data.keys()):
    text = test_data[key]['article']
    abstract = test_data[key]['abstract']
    abstract = ' '.join(abstract)
    result = model_orig(text)
    summary = ''.join(result)
    summaries[key] = {}
    summaries[key]['real'] = abstract
    summaries[key]['predicted'] = summary
    summaries[key]['rouge_score'] = scorer.score(abstract, summaries[key]['predicted'])
    scores['rouge1'].append(summaries[key]['rouge_score']['rouge1'].fmeasure)
    scores['rouge2'].append(summaries[key]['rouge_score']['rouge2'].fmeasure)
    scores['rougeLsum'].append(summaries[key]['rouge_score']['rougeLsum'].fmeasure)

print(sum(scores['rouge1']) / len(scores['rouge1']))
print(sum(scores['rouge2']) / len(scores['rouge2']))
print(sum(scores['rougeLsum']) / len(scores['rougeLsum']))
summaries['rouge1'] = sum(scores['rouge1']) / len(scores['rouge1'])
summaries['rouge2'] = sum(scores['rouge2']) / len(scores['rouge2'])
summaries['rougeLsum'] = sum(scores['rougeLsum']) / len(scores['rougeLsum'])


  2%|▏         | 22/1001 [08:13<6:05:42, 22.41s/it]


KeyboardInterrupt: 